In [11]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# from MCMAC_pre_merger import MCengine_pre_merger
# from MCMAC_post_merger import MCengine_post_merger 

# change prior: angle 0-45 degree
from modules.MCMAC_pre_merger_changeprior_45degree import MCengine_pre_merger
from modules.MCMAC_post_merger_changeprior_45degree import MCengine_post_merger 

import numpy as np
import pickle

# Input A1758 parameters

In [12]:
pre = False
post = True

# Setting simulation parameters (define prior distributions)
N_mc = 1000  # Number of Monte Carlo samples

change_prior_angle = True

cluster_name ='A1758'

if change_prior_angle is True:
    folder_name = cluster_name + '_0-45degree'
else:
    folder_name = cluster_name

arcsecond = 96
redshift = 0.2799

# (mean value, standard deviation)
M1 = (7.90e14, 1.89e14)  # Mass distribution of A1758NW (Subcluster 1)
M2 = (5.49e14, 1.67e14)  # Mass distribution of A1758NE (Subcluster 2)
Z1 = (0.2799, 0.0006)  # Redshift distribution of A1758NW (Subcluster 1)
Z2 = (0.2780, 0.0007)  # Redshift distribution of A1758NE (Subcluster 2)

TSC_simulation = 0.27

# when you use pre-MCMAC code
if pre is True:
    prefix = "../RealClusters/" + folder_name + "/simulation_pre_output" 

# when you use post-MCMAC code
if post is True:
    prefix = "../RealClusters/" + folder_name + "/simulation_post_output" 

result_file = "result_" + folder_name + ".csv"

## arcsecond to kpc

In [13]:
from astropy.cosmology import Planck18 as cosmo

def arcsecondToKpc(redshift, arcsecond):
    # Convert arcseconds to kiloparsecs
    kpc_per_arcsec = cosmo.kpc_proper_per_arcmin(redshift).value / 60  # kpc per arcsec
    distance_kpc = arcsecond * kpc_per_arcsec
    return distance_kpc

In [14]:

distance_Mpc = arcsecondToKpc(redshift, arcsecond) * 0.001 # kpc to Mpc

D_proj = (distance_Mpc, 0.05*distance_Mpc, 0.05*distance_Mpc)  # Projected separation distance distribution (mean 408 kpc, standard deviation 14 kpc)

## Confirm it collision or not

In [15]:
# Execute simulation (derive posterior distributions of dynamical properties such as collision velocity, time since collision, etc.)
# MCengine_pre_merger(N_mc, M1, M2, Z1, Z2, D_proj, prefix)

Probability of system being bound = 0.99200

Probability of system being boud is over 50%, then use post merger code.

In [16]:
MCengine_post_merger(N_mc, M1, M2, Z1, Z2, D_proj, prefix)

Completed Monte Carlo iteration 10 of 1000.
~3 minutes remaining
Completed Monte Carlo iteration 100 of 1000.
~3 minutes remaining
Completed Monte Carlo iteration 1000 of 1000.
~0 minutes remaining


(array([7.36107494e+14, 5.13172463e+14, 9.66077035e+14, 9.07274048e+14,
        5.94885237e+14, 5.86625215e+14, 9.56617250e+14, 8.50881144e+14,
        3.89150849e+14, 8.71121296e+14, 6.75124551e+14, 8.05718070e+14,
        9.12349688e+14, 6.26662518e+14, 9.14860402e+14, 9.14998741e+14,
        6.71370359e+14, 5.73860502e+14, 6.88290952e+14, 8.10749102e+14,
        9.90133564e+14, 3.59159591e+14, 7.65113826e+14, 4.00686137e+14,
        1.17974140e+15, 8.30450494e+14, 6.12652147e+14, 9.25029413e+14,
        9.72752031e+14, 8.09223017e+14, 8.57622451e+14, 7.90897178e+14,
        7.49584227e+14, 7.84513718e+14, 6.99360085e+14, 6.60078778e+14,
        5.81749792e+14, 5.74467816e+14, 8.11996599e+14, 8.80727945e+14,
        8.21579175e+14, 7.60242866e+14, 6.17809392e+14, 6.24181582e+14,
        7.24971144e+14, 1.13869385e+15, 1.05107218e+15, 8.47914744e+14,
        6.51620433e+14, 1.06461155e+15, 1.11946994e+15, 6.37244180e+14,
        9.29253491e+14, 7.22313293e+14, 5.87829508e+14, 7.085971

In [17]:
import pickle
import numpy as np
import pandas as pd

# 単位の定義
units = {
    'm1': '10^14 M☉',
    'm2': '10^14 M☉',
    'z1': '',
    'z2': '',
    'dproj': 'Mpc',
    'vradobs': 'km s^{-1}',
    'alpha': 'degree',
    'v3dobs': 'km s^{-1}',
    'd3d': 'Mpc',
    'v3dcol': 'km s^{-1}',
    'dmax': 'Mpc',
    'TSM0': 'Gyr',
    'TSM1': 'Gyr',
    'T': 'Gyr',
    'prob': ''
}

# パラメータファイルのリスト
parameters = ['_m_1.pickle', '_m_2.pickle', '_z_1.pickle', '_z_2.pickle', '_d_proj.pickle', 
              '_v_rad_obs.pickle', '_alpha.pickle', '_v_3d_obs.pickle', '_d_3d.pickle', 
              '_v_3d_col.pickle', '_d_max.pickle', '_TSM_0.pickle', '_TSM_1.pickle', 
              '_T.pickle', '_prob.pickle']

results = []

for param in parameters:
    with open(prefix + param, 'rb') as f:
        data = pickle.load(f)
        if param in ['_m_1.pickle', '_m_2.pickle']:
            # 質量をe14単位に変換
            data = np.array(data) / 1e14
        
        # Medianを計算
        median = np.median(data)
        
        # 68%および95%の信頼区間を計算
        lcl_68, ucl_68 = np.percentile(data, [16, 84])
        lcl_95, ucl_95 = np.percentile(data, [2.5, 97.5])
        
        param_name = param.replace('_', '').replace('.pickle', '')
        
        # 数値を有効数字2桁にフォーマット
        if param_name not in ['z_1', 'z_2']:
            median = round(median, 2)
            lcl_68 = round(lcl_68, 2)
            ucl_68 = round(ucl_68, 2)
            lcl_95 = round(lcl_95, 2)
            ucl_95 = round(ucl_95, 2)
        
        results.append({
            'Parameter': param_name,
            'Units': units[param_name],
            'Median': median,
            '68% LCL': lcl_68,
            '68% UCL': ucl_68,
            # '95% LCL': lcl_95,
            # '95% UCL': ucl_95
        })

# 結果をDataFrameに変換
df_results = pd.DataFrame(results)

display(df_results)

# TSM0とTSM1に対応する行の抽出
tsm0_row = df_results[df_results['Parameter'] == 'TSM0']
tsm1_row = df_results[df_results['Parameter'] == 'TSM1']

# CSVに出力するための新しい行を作成
new_row = {
    'ClusterName': cluster_name,
    'MCMAC_TSC0': tsm0_row['Median'].values[0],
    'MCMAC_TSC0.lower': tsm0_row['68% LCL'].values[0],
    'MCMAC_TSC0.upper': tsm0_row['68% UCL'].values[0],
    'MCMAC_TSC1': tsm1_row['Median'].values[0],
    'MCMAC_TSC1.lower': tsm1_row['68% LCL'].values[0],
    'MCMAC_TSC1.upper': tsm1_row['68% UCL'].values[0],
    'Simulation_TSC': TSC_simulation,
}

# 新しい行をDataFrameに変換
new_row_df = pd.DataFrame([new_row])

# CSVファイルに出力
new_row_df.to_csv(result_file, index=False)

,Parameter,Units,Median,68% LCL,68% UCL
0,m1,10^14 M☉,8.00,6.10,9.90
1,m2,10^14 M☉,5.70,4.02,7.29
2,z1,,0.28,0.28,0.28
3,z2,,0.28,0.28,0.28
4,dproj,Mpc,0.42,0.40,0.44
5,vradobs,km s^{-1},313.84,125.56,492.85
6,alpha,degree,9.57,5.62,12.17
7,v3dobs,km s^{-1},2173.41,1139.47,3012.08
8,d3d,Mpc,0.43,0.41,0.45
9,v3dcol,km s^{-1},2408.30,1536.32,3188.11
